# Kefir community simulation using SteadierCom

In this tutorial you will learn you use a community simulation tool to analyse
metabolic cross-feeding interactions in a microbial community.

Our case-study will be the kefir community from [Blasche et al, 2021](Blasche2021.pdf).

We will try to find potential cross-feeding interactions between the four most abundant species during milk fermentation:

- Lactobacillus kefiranofaciens
- Leuconocost mesenteroides
- Lactococcus lactis
- Acetobacter fabarum

![fig1bd](images/fig1bd.png)

We will use a new simulation tool called [SteadierCom](https://github.com/cdanielmachado/steadiercom) (still under development), that we discussed during the theoretical part of this course. 

## 1. Running SteadierCom

SteadierCom is a command-line tool. We can use command-line tools in Jupyter (as if we were typing directly on a terminal) by starting our code with `!`.

Let's start with testing the help menu:

In [1]:
! steadiercom -h

usage: steadiercom [-h] [-c COMMUNITIES.TSV] [-o OUTPUT] [-m MEDIA]
                   [--mediadb MEDIADB] [--growth GROWTH] [--sample SAMPLE]
                   [--we WE] [--wr WR] [--target TARGET] [--solver SOLVER]
                   [--unlimited UNLIMITED]
                   MODELS [MODELS ...]

Simulate microbial communities with SteadierCom

positional arguments:
  MODELS                
                        Multiple single-species models (one or more files).
                        
                        You can use wild-cards, for example: models/*.xml, and optionally protect with quotes to avoid automatic bash
                        expansion (this will be faster for long lists): "models/*.xml". 

options:
  -h, --help            show this help message and exit
  -c COMMUNITIES.TSV, --communities COMMUNITIES.TSV
                        
                        Run SteadierCom for multiple (sub)communities.
                        The communities must be specified in a ta

**SteadierCom** will require several inputs:

- Genome-scale metabolic models for each species
- Relative abundances of species (optional)
- Medium composition
- Community growth rate (optional)
- "Unlimited" compounds (like water, minerals, trace metals) to exclude from cross-feeding analysis (optional)
- Sample size (for sampling the solution space)

All these data has been prepared in the expected format (you can inspect it in the *data* folder). The growth rate and relative abundances have been grossly estimated from the figure above (assuming 25-fold total biomass increase in 90 hours).

We are now ready to run (this can take a few minutes):

In [2]:
! steadiercom models/*.xml -c data/abundance.tsv -m MILK --mediadb data/milk_composition.tsv --growth 0.05 --unlimited data/unlimited.txt --sample 100

simulating kefir in MILK medium


## 2. Analysing results

The output of **SteadierCom** is a table 

In [ ]:
df = pd.read_csv('output.tsv', sep='\t')

In [ ]:
df.query('donor == "L_kefiranofaciens" and receiver == "L_mesenteroides" and frequency > 0.1')

In [ ]:
df.query('receiver == "L_kefiranofaciens" and donor == "L_mesenteroides" and frequency > 0.1' )

In [ ]:
df.query('donor == "L_lactis" and receiver == "A_fabarum" and frequency > 0.1')

In [ ]:
df.query('donor == "A_fabarum" and receiver == "L_lactis" and frequency > 0.1')

In [ ]:
df.query("donor == 'environment'").groupby(['compound']).agg({'rate': sum}).sort_values('rate', ascending=False)

In [ ]:
from pyvis.network import Network

net = Network(directed=True, notebook=True, height='500px', width='800px')

selected = df.query("frequency > 0.1 and mass_rate > 0.001 and donor != 'environment' and receiver != 'environment'")

species = set(selected['donor']) | set(selected['receiver'])
net.add_nodes(species)

for cpd in set(selected['compound']):
    net.add_node(cpd, shape='box')

for _, row in selected.iterrows():
    net.add_edge(row['donor'], row['compound'], value=row['mass_rate'])
    net.add_edge(row['compound'], row['receiver'], value=row['mass_rate'])

net.show('network.html')

### SMETANA

In [ ]:
df2 = pd.read_csv('detailed.tsv', sep='\t')

In [ ]:
df2.query('donor == "L_kefiranofaciens" and receiver == "L_mesenteroides" and smetana > 0.1')

In [ ]:
df2.query('receiver == "L_kefiranofaciens" and donor == "L_mesenteroides" and smetana > 0.1' )

In [ ]:
df2.query('donor == "L_lactis" and receiver == "A_fabarum" and smetana > 0.1')

In [ ]:
from pyvis.network import Network

net = Network(directed=True, notebook=True, height='500px', width='800px')

selected = df2.query("smetana > 0.1")

species = set(selected['donor']) | set(selected['receiver'])
net.add_nodes(species)

for cpd in set(selected['compound']):
    net.add_node(cpd, shape='box')

for _, row in selected.iterrows():
    net.add_edge(row['donor'], row['compound'], value=row['smetana'])
    net.add_edge(row['compound'], row['receiver'], value=row['smetana'])

net.show('network2.html')